# cs8321 Lab 2 - Transfer Learning and Transformers

#### Chip Henderson - 48996654

## [2.0 points] Dataset Overview

**What is the classification task**

**What is the feature data? Who collected the data? Why? When? Is it multimodal?**

**What evaluation criteria will you be using, why?**

## [2.0 points] Describe the foundational model that you will be using to transfer learn from

**What tasks was the foundational model trained from?**

Per the hugging face [blog site](https://huggingface.co/google-bert/bert-base-uncased?text=The+goal+of+a+dog%27s+life+is+%5BMASK%5D), "the BERT model was pretrained on BookCorpus, a dataset consisting of 11,038 unpublished books and English Wikipedia. It is also known as the Toronto Book Corpus, and consists of the text of around 7,000 self-published books scraped from the indie ebook distribution website Smashwords [per wikipedia](https://en.wikipedia.org/wiki/BookCorpus). The dataset consists of around 985 million words across a large span of genres.

**Explain if the new task is within the same domain, across domains, etc.**

## [1.0 points] Split the data into training and testing

In [1]:

import tensorflow as tf
import numpy as np



In [2]:
%time
# Source: modified from https://nijianmo.github.io/amazon/index.html for importing data
# Customized path and df name 
import pandas as pd
import gzip
import json

path = "../Data_sources/Electronics_5.json.gz"

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF(path)


CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
df.head(10)

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,67,True,"09 18, 1999",AAP7PPBU72QFM,0151004714,{'Format:': ' Hardcover'},D. C. Carrad,This is the best novel I have read in 2 or 3 y...,A star is born,937612800,NaN
1,3.0,5,True,"10 23, 2013",A2E168DTVGE6SV,0151004714,{'Format:': ' Kindle Edition'},Evy,"Pages and pages of introspection, in the style...",A stream of consciousness novel,1382486400,NaN
2,5.0,4,False,"09 2, 2008",A1ER5AYS3FQ9O3,0151004714,{'Format:': ' Paperback'},Kcorn,This is the kind of novel to read when you hav...,I'm a huge fan of the author and this one did ...,1220313600,NaN
3,5.0,13,False,"09 4, 2000",A1T17LMQABMBN5,0151004714,{'Format:': ' Hardcover'},Caf Girl Writes,What gorgeous language! What an incredible wri...,The most beautiful book I have ever read!,968025600,NaN
4,3.0,8,True,"02 4, 2000",A3QHJ0FXK33OBE,0151004714,{'Format:': ' Hardcover'},W. Shane Schmidt,I was taken in by reviews that compared this b...,A dissenting view--In part.,949622400,NaN
5,4.0,NaN,True,"06 5, 2013",A3IYSOTP3HA77N,0380709473,{'Format:': ' Kindle Edition'},B. Marks,I read this probably 50 years ago in my youth ...,Above average mystery,1370390400,NaN
6,5.0,NaN,True,"06 27, 2016",A11SXV34PZUQ5E,0380709473,{'Format:': ' Kindle Edition'},Tom C.,I read every Perry mason book voraciously. Fin...,Lam is cool!,1466985600,NaN
7,5.0,NaN,True,"07 30, 2015",A2AUQM1HT2D5T8,0380709473,{'Format:': ' Kindle Edition'},ema,I love this series of Bertha and Lamb.. Great...,Five Stars,1438214400,NaN
8,5.0,NaN,True,"02 16, 2015",A3UD8JRWLX6SRX,0380709473,{'Format:': ' Paperback'},Michael O.,Great read!,Five Stars,1424044800,NaN
9,4.0,NaN,False,"11 21, 2013",A3MV1KKHX51FYT,0380709473,{'Format:': ' Paperback'},Acute Observer,"Crows Can't Count, A.A. Fair\n\nMr. Harry Shar...",A Fast and Far Moving Adventure,1384992000,NaN


Strangely, these don't all seem like electronics reviews...but for my purposes it really doesn't matter.

**explain how you performed this operation and why you think it is reasonable to split this particular dataset this way**

**For multi-task datasets, be sure to explain if it is appropriate to stratify within each task.**

**If the dataset is already split for you, explain how the split was achieved and how it is stratified.**

## [2.0 points] Train a model from scratch to perform the classification task (this does NOT need to be a transformer). 

**Verify the model converges (even if the model is overfit).**

#### Convolutional Neural Network 1 (CNN-1)

The first CNN I'll run will consist of 64 filters with a width of 5. I'm changing the filter size from the in-class example because my dataset is quite a bit smaller. So my thought being I won't need so many filters to get good results.

In [5]:
if False:
    # Source: Modified from in-class lecture, cs7324, notebook 13a
    from tensorflow.keras.metrics import Precision

    EMBED_SIZE = 50  # same size as loaded from GLOVE
    NUM_CLASSES = 3
    sequence_input = Input(shape=(MAX_TWEET_LEN,), dtype='int32')
    # starting size: 500
    embedded_sequences = embedding_layer(sequence_input) # from previous embedding
    x = Conv1D(64, 5, activation='relu',
            kernel_initializer='he_uniform')(embedded_sequences)

    # after conv, size becomes: 500-4=496
    x = MaxPooling1D(5)(x) # after max pool, 996/5 = 99
    x = Dropout(0.2)(x) # after dropout, size is 95
    x = Conv1D(64, 5, activation='relu',
            kernel_initializer='he_uniform')(x)

    # new size is 195
    x = MaxPooling1D(5)(x) # after max pool, size is 95/5 = 19
    x = Dropout(0.2)(x)
    x = Conv1D(64, 5, activation='relu',
            kernel_initializer='he_uniform')(x)

    # after convolution, size becomes 15 elements long
    x = MaxPooling1D(5)(x) # this is the size to globally flatten, 15/5 = 3
    # flattened vector max pools across each of the 3 elements
    # so vectors is now 192 dimensions 3*64 = 192
    x = Flatten()(x)

    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu',
            kernel_initializer='he_uniform')(x)

    preds = Dense(NUM_CLASSES, activation='softmax',
                kernel_initializer='glorot_uniform')(x)

    model_cnn_1 = Model(sequence_input, preds)

    # if representing as OHE, use categorical_crossentropy
    # if representing the class as an integer, use sparse_categorical_crossentropy
    model_cnn_1.compile(loss='categorical_crossentropy', 
                optimizer='rmsprop',
                metrics=['Precision'])

    print(model_cnn_1.summary())

    cnn1_histories = []
    tmp = model_cnn_1.fit(X_train, y_train_ohe, validation_data=(X_test, y_test_ohe),
            epochs=30, batch_size=128)
    cnn1_histories.append(tmp)

## [2.0 points] Transfer Learning

In [6]:
%time
from transformers import BertTokenizer, TFBertModel

CPU times: total: 0 ns
Wall time: 0 ns


**Train a model by transfer learning from your foundational model.**

**Verify that the new model converges. You only need to train a model using the bottleneck features for this step.**

## [2.0 points] Fine Tuning

**Perform fine tuning upon the model by training some layers within the foundational model**

**Verify that the model converges.**

## [4.0 points] Report the results

**Report the results of all models using the evaluation procedure that you argued for at the beginning of the lab.**

**Compare the convergence of the models and the running time.**

**Results should be reported with proper statistical comparisons and proper visualizations.**